In [1]:
import asyncio
import os
from dotenv import load_dotenv
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.ui import Console
from azure.core.credentials import AzureKeyCredential
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken

from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console


In [2]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

finish_reason='stop' content='The capital of France is Paris.' usage=RequestUsage(prompt_tokens=14, completion_tokens=8) cached=False logprobs=None thought=None


In [4]:
import nest_asyncio, asyncio

# Prompt the user for the trip request and run the multi‑agent workflow

nest_asyncio.apply()  # allow nested event loops inside Jupyter


async def book_trip_interactive() -> None:
    user_task = input("Please describe the trip you'd like to book: ")

    async def lookup_hotel(location: str) -> str:
        return f"Here are some hotels in {location}: hotel1, hotel2, hotel3."

    async def lookup_flight(origin: str, destination: str) -> str:
        return f"Here are some flights from {origin} to {destination}: flight1, flight2, flight3."

    async def book_trip() -> str:
        return "Your trip is booked!"

    travel_advisor = AssistantAgent(
        "Travel_Advisor",
        model_client=client,
        tools=[book_trip],
        description="Helps with travel planning.",
    )
    hotel_agent = AssistantAgent(
        "Hotel_Agent",
        model_client=client,
        tools=[lookup_hotel],
        description="Helps with hotel booking.",
    )
    flight_agent = AssistantAgent(
        "Flight_Agent",
        model_client=client,
        tools=[lookup_flight],
        description="Helps with flight booking.",
    )

    termination = TextMentionTermination("TERMINATE")
    team = SelectorGroupChat(
        [travel_advisor, hotel_agent, flight_agent],
        model_client=client,
        termination_condition=termination,
    )

    # Stream the multi‑agent chat to the notebook console
    await Console(team.run_stream(task=user_task))  # type: ignore
    # Give the user the option to start a new chat or end the session
    while True:
        user_task = input("Please describe the trip you'd like to book: ")
        if user_task.lower() == "exit":
            break
        await Console(team.run_stream(task=user_task))  # type: ignore
        
        
# Run the interactive trip booking
await book_trip_interactive()

---------- user ----------
3 day trip to rome
---------- Travel_Advisor ----------
[FunctionCall(id='call_5zYwjTV9nt6coIYXNRhDczbx', arguments='{}', name='book_trip')]
---------- Travel_Advisor ----------
[FunctionExecutionResult(content='Your trip is booked!', call_id='call_5zYwjTV9nt6coIYXNRhDczbx')]
---------- Travel_Advisor ----------
Your trip is booked!
---------- Hotel_Agent ----------
Thank you! If you need any assistance with your trip to Rome, such as finding accommodations, attractions, or restaurants, just let me know!
---------- Flight_Agent ----------
TERMINATE
